In [ ]:
import pydicom
import pandas as pd
from matplotlib import pyplot
import cv2 
import random
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [ ]:
def junta(lista):
    listaJunto=[]
    locs=[]
    nome=''
    for ind, linha in lista.iterrows():
        if nome == linha['patientId']:
            listaJunto[-1][2].append({'x':linha['x'],'y':linha['y'], 'width':linha['width'], 'height':linha['height']})    
            
        else:
            nome = linha['patientId']
            target = linha['Target']
            if target == 1:
                locs.append({'x':linha['x'],'y':linha['y'], 'width':linha['width'], 'height':linha['height']}) 
            
            listaJunto.append([nome, target, locs])
            locs=[]
    
    return listaJunto

    

In [ ]:
trainImg = []
img =[]

tamanho =  (256,256)
kernel = (16,16) #x,y
meioKernel = (kernel[0]/2,kernel[1]/2)
factorX = 1024/tamanho[0]
factorY = 1024/tamanho[1]

treino = pd.read_csv('../input/rsna-pneumonia-detection-challenge/stage_1_train_labels.csv')
treino = junta(treino)

#image proc
def imProc(imag):    
    imag= cv2.convertScaleAbs(imag, alpha=3.5, beta=-500)
    imag = cv2.resize(imag, tamanho)
    imag = imag/255.0
    return(imag)    

for i in range(0, 10000):
    DICOM = pydicom.read_file('../input/rsna-pneumonia-detection-challenge/stage_1_train_images/{}.dcm'.format(treino[i][0]))
    img.append(DICOM)  

graf, loc= pyplot.subplots(2,2, figsize=(15,15))

#temp = cv2.rectangle(temp, (31,59),(204,168),1 )
loc[0,0].imshow(img[4].pixel_array)
loc[0,0].set_title(treino[4][2])

loc[0,1].imshow(trainArray[148153])
loc[0,1].set_title(targetArray[148153])

temp = trainProc[4].copy()
#temp = cv2.rectangle(temp, (145,81),(169 ,119),1 )
temp = cv2.rectangle(temp, (81, 145),(119, 169),1 )

loc[1,0].imshow(temp)
loc[1,0].set_title(treino[7][1])

temp = img[4].pixel_array.copy()
x= int(treino[4][2][0]['x'])
y=int(treino[4][2][0]['y'])
y2= int(treino[4][2][0]['y']+treino[4][2][0]['height'])
x2= int(treino[4][2][0]['x']+treino[4][2][0]['width'] )    

temp = cv2.rectangle(temp, (x,y) ,(x2, y2), 255 )
loc[1,1].imshow(temp)
print(x,y,x2,y2)


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('ml4_cp.hdf5', verbose=1, save_best_only=True)
epocas = 1
#
#model = keras.Sequential([
#    keras.layers.Conv2D(6, kernel_size=(3,3), strides=(1, 1), activation='relu', input_shape=(kernel[0], kernel[1],1)),
#    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2) ),
#   # keras.layers.Conv2D(13, kernel_size=(3,3), strides=(1, 1), activation='relu', input_shape=(20,14,1)),
#   # keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2) ),
#    keras.layers.Flatten(),
# #   keras.layers.Dense(140, activation=tf.sigmoid),
#    keras.layers.Dense(70, activation=tf.sigmoid),
#    keras.layers.Dense(2, activation=tf.nn.softmax)
#])

#when ready#
model = keras.models.load_model('../input/pulm-o-ml-4/ml4.h5')
model.load_weights('../input/pulm-o-ml-4/ml4_w.h5')
#model.load_weights('../input/pulm-o-ml-4/ml4_cp.hdf5')
####

model.compile(optimizer= keras.optimizers.SGD(lr=0.01, momentum=0, decay=0, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


Repeating to all the dataset

In [ ]:
batch = 5
ini =0
fim = 55
epocas = 1
limite = 60
while(fim < limite):
    ini=fim
    fim = fim + batch
    if(fim > limite):
        fim = limite
    print(ini, fim)

    img =[]
    treinoInfo=[]
    
    for i in range(ini, fim):
        DICOM = pydicom.read_file('../input/rsna-pneumonia-detection-challenge/stage_1_train_images/{}.dcm'.format(treino[i][0]))
        img.append(DICOM) 
        treinoInfo.append(treino[i])
    
    #processando as imagens
    trainProc=[]  
    for i in range(0, batch):
        temp = img[i].pixel_array.copy()
        temp= imProc(temp)
        trainProc.append(temp)

    #extraindo os valores positivos e negativos
    trainArray=[]
    targetArray=[]
    for i in range(0, batch):
    #l=linha  c=coluna; p = casos positivos
        if treinoInfo[i][1] == 1:        
            for l in range(0, tamanho[0]-kernel[0]):
                for c in range(0, tamanho[1]-kernel[1]):
                #adicionando o elemento
                    alt = l+kernel[0]
                    larg= c+kernel[1]
                    trainArray.append(trainProc[i][l:alt, c:larg])
                
                #verificando se é positivo ou não 
                    positivo =False
                    for coord in treinoInfo[i][2]:                                       
                        px= int(coord['x']/factorX)
                        py= int(coord['y']/factorY)
                        px2= int(px+(coord['width']/factorX))
                        py2 = int(py+(coord['height']/factorY)) 
                        lMeio = int(l+meioKernel[0])
                        cMeio = int(c+meioKernel[1])
                        if ((py < lMeio ) and (lMeio <  py2)):
                            if((px < cMeio) and (cMeio <  px2)):
                                positivo = True                 
                            
                    if positivo == True:
                        targetArray.append(1)
                    else:
                        targetArray.append(0)

        else:
            for l in range(0, tamanho[0]-kernel[0]):
                for c in range(0, tamanho[1]-kernel[1]):
                    alt = l+kernel[0]
                    larg= c+kernel[1]         
                    trainArray.append(trainProc[i][l:alt , c:larg])
                    targetArray.append(0)
                
    img = None    
    npImg = np.array(trainArray)
    trainArray = None

    npTarget =  np.array(targetArray)
    targetArray = None

    trainIndex, testIndex = train_test_split(range(0, len(npTarget)) , test_size= 0.3) 
    #--
    train= npImg[trainIndex].reshape([-1, kernel[0], kernel[1],1])
    test = npImg[testIndex].reshape([-1,kernel[0], kernel[1],1])
    #--
    train = tf.convert_to_tensor(train)
    trainTarget = tf.one_hot(npTarget[trainIndex],2)
    test = tf.convert_to_tensor(test)
    testTarget = tf.one_hot(npTarget[testIndex],2)
    model.fit(train, trainTarget, epochs=epocas, steps_per_epoch=int(len(trainIndex)/100), 
          validation_data=(test, testTarget), validation_steps=int(len(testIndex)/100),
          callbacks=[checkpoint])

In [ ]:
print(fim)

In [ ]:
path='ml4.h5'
model.save_weights('ml4_w.h5')
model.save(path)
